In [1]:
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import MmCorpus
from gensim.corpora import Dictionary
from collections import Counter, defaultdict
from numpy.random import seed
import re
from metadata.metadata import ACL_metadata
from _topic_modeling.lda_loader import Loader
import _pickle as pkl
from metadata import Gender
import matplotlib.pyplot as plt
import numpy as np
import logging
import gensim 
seed(1)
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [2]:
loader = Loader()

loaded corpus index from /home/ramona/Desktop/Honours-LDA/aan/../acl_bow8.mm.index
initializing corpus reader from /home/ramona/Desktop/Honours-LDA/aan/../acl_bow8.mm
accepted corpus with 22405 documents, 10000 features, 9693790 non-zero entries


In [3]:
lda = gensim.models.ldamodel.LdaModel.load('../lda8_gensim_100it_1000passes_10kcorpus.lda')
#lda = gensim.models.ldamodel.LdaModel.load('../models/model500.pkl')

loading LdaModel object from ../lda8_gensim_100it_1000passes_10kcorpus.lda
loading expElogbeta from ../lda8_gensim_100it_1000passes_10kcorpus.lda.expElogbeta.npy with mmap=None
setting ignored attribute id2word to None
setting ignored attribute dispatcher to None
setting ignored attribute state to None
loaded ../lda8_gensim_100it_1000passes_10kcorpus.lda
loading LdaModel object from ../lda8_gensim_100it_1000passes_10kcorpus.lda.state
loaded ../lda8_gensim_100it_1000passes_10kcorpus.lda.state


In [4]:
lda.id2word = loader.id2word

In [5]:
lda.get_document_topics(loader.corpus[1])

[(15, 0.018853865874859026),
 (23, 0.050862754789929389),
 (25, 0.015930458374510091),
 (31, 0.061798693485785816),
 (45, 0.075257759619355005),
 (49, 0.089543908211750511),
 (62, 0.026627839640632464),
 (65, 0.10298193141460529),
 (86, 0.013822900620438441),
 (87, 0.21678643256705096),
 (88, 0.06477404276506836),
 (89, 0.013421507337851067),
 (96, 0.010108148030084028),
 (99, 0.14925223092989603)]

In [6]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [7]:
len(loader.id2word.keys())

10000

In [6]:
corpus = loader.corpus
dictionary = loader.dic

In [9]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/home/ramona/anaconda3/envs/mlp/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [10]:
gensimcm = CoherenceModel(model=lda, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [13]:
gensimcm.get_coherence()

CorpusAccumulator accumulated stats from 1000 documents
CorpusAccumulator accumulated stats from 2000 documents
CorpusAccumulator accumulated stats from 3000 documents
CorpusAccumulator accumulated stats from 4000 documents
CorpusAccumulator accumulated stats from 5000 documents
CorpusAccumulator accumulated stats from 6000 documents
CorpusAccumulator accumulated stats from 7000 documents
CorpusAccumulator accumulated stats from 8000 documents
CorpusAccumulator accumulated stats from 9000 documents
CorpusAccumulator accumulated stats from 10000 documents
CorpusAccumulator accumulated stats from 11000 documents
CorpusAccumulator accumulated stats from 12000 documents
CorpusAccumulator accumulated stats from 13000 documents
CorpusAccumulator accumulated stats from 14000 documents
CorpusAccumulator accumulated stats from 15000 documents
CorpusAccumulator accumulated stats from 16000 documents
CorpusAccumulator accumulated stats from 17000 documents
CorpusAccumulator accumulated stats from

-1.6901845123922141

In [11]:
gensimcm.get_coherence()

CorpusAccumulator accumulated stats from 1000 documents
CorpusAccumulator accumulated stats from 2000 documents
CorpusAccumulator accumulated stats from 3000 documents
CorpusAccumulator accumulated stats from 4000 documents
CorpusAccumulator accumulated stats from 5000 documents
CorpusAccumulator accumulated stats from 6000 documents
CorpusAccumulator accumulated stats from 7000 documents
CorpusAccumulator accumulated stats from 8000 documents
CorpusAccumulator accumulated stats from 9000 documents
CorpusAccumulator accumulated stats from 10000 documents
CorpusAccumulator accumulated stats from 11000 documents
CorpusAccumulator accumulated stats from 12000 documents
CorpusAccumulator accumulated stats from 13000 documents
CorpusAccumulator accumulated stats from 14000 documents
CorpusAccumulator accumulated stats from 15000 documents
CorpusAccumulator accumulated stats from 16000 documents
CorpusAccumulator accumulated stats from 17000 documents
CorpusAccumulator accumulated stats from

-1.7499231027069384

In [8]:
acl = ACL_metadata()
df = acl.modeling_df

Remember to use acl.modeling_files and modeling_df for topic modeling


In [9]:
for i, p in lda.get_topic_terms(38):
    print(loader.id2word[i])

loria
rectness
sort
itute
bootstrapp
eld
guage_technology
label_unlabeled
pursue
telecommunication


In [10]:
#doc_topics = list(map(lambda x: lda.get_document_topics(x), corpus))

In [11]:
# with open("../doc_topics_gensim8.pkl","wb") as f:
#     pkl.dump(doc_topics, f)
with open("../doc_topics_gensim8.pkl","rb") as f:
    doc_topics = pkl.load(f)

In [12]:
dids = loader.doc_ids
len(dids)

22405

In [13]:
females_topics = []
females_topics_proportions = defaultdict(int) # P(topic|gender)'
males_topics_proportions = defaultdict(int)
fem_norm = 0 # normalizer
male_norm = 0
males_topics = []
for i, file in enumerate(acl.modeling_files):
    try:
        topics = doc_topics[i]
        gender = df.loc[acl.get_id(file)]["genders"][0] #first author
        if gender == Gender.female:
            for topic, p in topics:
                females_topics_proportions[topic] += p
            fem_norm +=1
        elif gender == Gender.male:
            for topic, p in topics:
                males_topics_proportions[topic] += p
            male_norm +=1
    except KeyError:
        pass

In [14]:
doc_topics[21]

[(0, 0.020886336519849568),
 (7, 0.020881526047643453),
 (9, 0.020924753807451915),
 (11, 0.060986149778976172),
 (18, 0.020381093520146651),
 (31, 0.075322132236959966),
 (45, 0.011625826332583118),
 (49, 0.11748712931281058),
 (50, 0.063636055603021463),
 (54, 0.012365581858891013),
 (58, 0.016056874388201661),
 (61, 0.043981281565683004),
 (69, 0.043445186777176223),
 (70, 0.012110003567150779),
 (75, 0.11935327056809771),
 (78, 0.018921869113265502),
 (83, 0.037206462066415417),
 (88, 0.031864086815699043),
 (91, 0.077150370245594735),
 (95, 0.037267188865083865),
 (99, 0.0345907763174476)]

In [15]:
for idx in range(100):
    f = (females_topics_proportions[idx] + 1e-10) / fem_norm
    m = (males_topics_proportions[idx] + 1e-10) / male_norm
    odds = (f * (1-f)) / (m * (1-m))
    if odds > 1.5:
        print("Females are {0:.2f} more likeky than men to write about topic {1:d}".format(odds, idx))
        print("Most common words for this topic are:")
        for w,p in lda.show_topic(idx,10):
            print(w,)
        print("-"*30)
        print("-"*30)


Females are 1.55 more likeky than men to write about topic 2
Most common words for this topic are:
experimental
relationship
clinical
partition
fail
modifier
exception
transitive_intransitive
query
full
------------------------------
------------------------------
Females are 1.76 more likeky than men to write about topic 3
Most common words for this topic are:
taipei_taiwan
tag
argument
diverse_range
systematically
rence
sketch
energy
grammar_ule
hope
------------------------------
------------------------------
Females are 1.60 more likeky than men to write about topic 48
Most common words for this topic are:
teractive
symbol
greement
idea
individual
writing_assistance
tailor
stand
roughly
site
------------------------------
------------------------------
Females are 1.58 more likeky than men to write about topic 60
Most common words for this topic are:
annual_conference
side
clear
nanba
marseille
mat
hold
bois
disguise
morphology_phonology
------------------------------
------------

In [16]:
for idx in range(100):
    f = (females_topics_proportions[idx] + 1e-10) / fem_norm
    m = (males_topics_proportions[idx] + 1e-10) / male_norm
    odds = (m * (1-m)) / (f * (1-f))
    if odds > 1.5:
        print("Males are {0:.2f} more likeky than females to write about topic {1:d}".format(odds, idx))
        print("Most common words for this topic are:")
        for w,p in lda.show_topic(idx,10):
            print(w,)
        print("-"*30)
        print("-"*30)


Males are 1.74 more likeky than females to write about topic 34
Most common words for this topic are:
teraction
indication
intersection
shoe
initial
table_summarize
session_page
conference_intelligent
darpa_workshop
error_prone
------------------------------
------------------------------
Males are 1.86 more likeky than females to write about topic 39
Most common words for this topic are:
omit
cally
emnlp_page
finite
alternation
invade
intersect
insertion
initiative_administer
percentage
------------------------------
------------------------------
Males are 1.56 more likeky than females to write about topic 56
Most common words for this topic are:
california
disorder
question
homogeneous
adverbial_clause
bobrow_collin
reflect
adduce
proliferate
hannan
------------------------------
------------------------------
Males are 1.59 more likeky than females to write about topic 62
Most common words for this topic are:
processor
clause
unique
clean
compositional
cis
partir
ous
hardware
probl

In [8]:
with open("../topic-terms.txt","r") as f:
    topics = f.read().split("\n")

In [9]:
j_doc_topics = defaultdict(list)
j_dic = gensim.corpora.Dictionary()
vv = list()
for topic in topics:
    if len(topic) > 10:
        number, name, words = topic.split(":")
        t = words.strip()
        tt = list(map(lambda x: x.strip(), re.split("\\[0\\.\\d\\d\\]",t)))
        if len(tt) == 151:
            tt = tt
            j_doc_topics[number + name] = tt
            j_dic.add_documents([tt])
            vv.append(tt)

adding document #0 to Dictionary(0 unique tokens: [])
built Dictionary(151 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 1 documents (total 151 corpus positions)
adding document #0 to Dictionary(151 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(293 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 2 documents (total 302 corpus positions)
adding document #0 to Dictionary(293 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(431 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 3 documents (total 453 corpus positions)
adding document #0 to Dictionary(431 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(547 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 4 documents (total 604 corp

built Dictionary(2505 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 31 documents (total 4681 corpus positions)
adding document #0 to Dictionary(2505 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(2555 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 32 documents (total 4832 corpus positions)
adding document #0 to Dictionary(2555 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(2611 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 33 documents (total 4983 corpus positions)
adding document #0 to Dictionary(2611 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(2653 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 34 documents (total 5134 corpus positions)
adding document #0 to Dic

built Dictionary(4149 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 61 documents (total 9211 corpus positions)
adding document #0 to Dictionary(4149 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(4197 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 62 documents (total 9362 corpus positions)
adding document #0 to Dictionary(4197 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(4238 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 63 documents (total 9513 corpus positions)
adding document #0 to Dictionary(4238 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(4258 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 64 documents (total 9664 corpus positions)
adding document #0 to Dic

built Dictionary(5390 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 91 documents (total 13741 corpus positions)
adding document #0 to Dictionary(5390 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(5413 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 92 documents (total 13892 corpus positions)
adding document #0 to Dictionary(5413 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(5446 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 93 documents (total 14043 corpus positions)
adding document #0 to Dictionary(5446 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...)
built Dictionary(5457 unique tokens: ['resolution', 'pronoun', 'anaphora', 'antecedent', 'pronouns']...) from 94 documents (total 14194 corpus positions)
adding document #0 to

In [10]:
j_topics_bow = defaultdict(list)
a = []
for i, jbow in j_doc_topics.items():
    j_topics_bow[i] = j_dic.doc2bow(jbow)
    a.append(j_dic.doc2bow(jbow))

In [11]:
r_doc_topics = defaultdict(list)
for t_n in range(100):
    for i, p in lda.get_topic_terms(t_n,151):
        r_doc_topics[t_n].append(loader.id2word[i])

In [12]:
j_dic

In [13]:
tfidf = gensim.models.TfidfModel(a)

collecting document frequencies
PROGRESS: processing document #0
calculating IDF weights for 100 documents and 5576 features (15100 matrix non-zeros)


In [15]:
index = gensim.similarities.SparseMatrixSimilarity(tfidf[corpus] , num_features=5576)

creating sparse index
creating sparse matrix from corpus
PROGRESS: at document #0
PROGRESS: at document #10000
PROGRESS: at document #20000
created <22405x5576 sparse matrix of type '<class 'numpy.float32'>'
	with 8194237 stored elements in Compressed Sparse Row format>


In [16]:
index

In [ ]:
index[j_dic.doc2bow(r_doc_topics[2])]

In [ ]:
from scipy import spatial
corresp = defaultdict()
for i,r in r_doc_topics.items():
    r_2bow = j_dic.doc2bow(r)
    min_distance = float('inf')
    sims = index[r_2bow]
    best_name = ""
#     for name, jt in j_topics_bow.items():
#         result = gensim.matutils.cossim(r_2bow, jt)
#         #result = 1 - spatial.distance.cosine(r_2bow, jt)
#         if(result) < min_distance:
#             best_name = name
#             min_distance = result
#     corresp[i] = best_name
#     print(best_name, lda.show_topic(i,5))

In [15]:
corresp

defaultdict(None,
            {0: 'Topic 34 #random',
             1: 'Topic 34 #random',
             2: 'Topic 34 #random',
             3: 'Topic 34 #random',
             4: 'Topic 15 #POSTagging',
             5: 'Topic 34 #random',
             6: 'Topic 2 #random(suffixesprefixes)',
             7: 'Topic 34 #random',
             8: 'Topic 5 #GraphTheory+BioNLP',
             9: 'Topic 34 #random',
             10: 'Topic 34 #random',
             11: 'Topic 14 #SpellCorrection',
             12: 'Topic 34 #random',
             13: 'Topic 34 #random',
             14: 'Topic 4 #LinguisticExampleSentences',
             15: 'Topic 7 #ConceptOntologies/KnowledgeRep',
             16: 'Topic 29 #ngramLanguageModels',
             17: 'Topic 12 #random(numbers)',
             18: 'Topic 7 #ConceptOntologies/KnowledgeRep',
             19: 'Topic 5 #GraphTheory+BioNLP',
             20: 'Topic 34 #random',
             21: 'Topic 34 #random',
             22: 'Topic 12 #random(numb

In [180]:
lda.show_topic(93,10)

[('finite', 0.075826165549581506),
 ('transducer', 0.055276832995990989),
 ('finite_state', 0.049500502558631657),
 ('transition', 0.049183144334694472),
 ('regular', 0.041894185014314946),
 ('automaton', 0.029666310624355017),
 ('symbol', 0.024545344280760901),
 ('string', 0.024034936500403493),
 ('fst', 0.018303242416231363),
 ('automata', 0.015468168735721828)]

In [176]:
lda.show_topic(92,10)

[('lattice', 0.041838803414931271),
 ('decoding', 0.030266187259806038),
 ('prune', 0.028713988997792733),
 ('beam', 0.027085387706856128),
 ('decoder', 0.022642335195209137),
 ('decode', 0.018703364313871346),
 ('pruning', 0.014777404810311033),
 ('synchronous', 0.013759753855595042),
 ('hierarchical', 0.01270116671082798),
 ('derivation', 0.01167343313090918)]